# Image Analysis Agent with GCS Artifact Persistence and Vertex AI Agent Engine Deployment

This notebook demonstrates how to build and deploy an image analysis application using the **Google Agent Development Kit (ADK)** and **Vertex AI Agent Engine**. It specifically showcases how to handle persistent image data using **Google Cloud Storage (GCS)** as an artifact service.

---

### Key Components

* **LlmAgent (Root Agent):** Configured with `gemini-2.5-flash`. It is given specific instructions to analyze images and is equipped with the `load_artifacts` tool to retrieve image content when it isn't directly in the prompt context.
* **GcsArtifactService:** Manages the storage of files in a GCS bucket. This allows the application to handle large files (like 4K images) or multiple files without bloating the immediate chat history.
* **SaveFilesAsArtifactsPlugin:** An ADK plugin that automatically intercepts files sent in a message and saves them as managed artifacts in the GCS bucket.
* **AdkApp:** The core application wrapper that integrates the agent, the artifact service, and the plugins into a single deployable unit.

### Workflow Summary

1. **Local Execution and Artifact Management:**
* **Initialization:** The user sends a query with a base64-encoded image (inline data) to the `AdkApp`.
* **Interception:** The `SaveFilesAsArtifactsPlugin` automatically intercepts the image data and persists it into the configured **Google Cloud Storage (GCS)** bucket.
* **Tool Usage:** The `LlmAgent` (powered by `gemini-2.5-flash`) identifies that it needs to see the image and invokes the `load_artifacts` tool to retrieve the content.
* **History Optimization:** By inspecting the session events, the notebook demonstrates that the chat history contains only artifact metadata and file URIs, not the raw image binaries, preventing session bloat.


2. **Multi-turn Reasoning and Context Caching:**
* **Follow-up Queries:** In subsequent turns (e.g., asking about dogs in the "previous image"), the agent continues to use the `load_artifacts` tool to maintain visual context.
* **Efficiency:** The execution logs reveal the use of **Vertex AI Context Caching**, which allows the model to refer back to previously loaded image data efficiently without re-processing the entire file in every turn.


3. **Remote Deployment to Agent Engine:**
* **Deployment:** The `AdkApp` is packaged and deployed to **Vertex AI Agent Engine** as a remote service.
* **Validation:** The notebook verifies the remote deployment by performing the same multi-modal analysis (describing the image and answering follow-up questions) against the cloud-hosted instance, confirming that GCS persistence and tool-calling work seamlessly in a production-like environment.

---

### Why this architecture matters

By using `GcsArtifactService`, you avoid passing massive amounts of raw data back and forth in every turn of a conversation. Instead, the data stays in storage, and the agent only "loads" what it needs, making the application more scalable and cost-effective for multi-modal tasks.

## Install packages

In [ ]:
%pip install --upgrade --user google-adk

In [ ]:
# Reboot kernel
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Preparation

In [1]:
import base64
import os
import vertexai
from vertexai.agent_engines import AdkApp
from google.adk.agents import LlmAgent
from google.adk.artifacts import GcsArtifactService
from google.adk.planners import BuiltInPlanner
from google.adk.plugins.save_files_as_artifacts_plugin import SaveFilesAsArtifactsPlugin
from google.adk.tools import load_artifacts
from google.genai.types import Part, Content, Blob, ThinkingConfig

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

BUCKET = f'{PROJECT_ID}_artifacts'
!gsutil ls -b gs://{BUCKET} 2>/dev/null || \
 gsutil mb -l {LOCATION} gs://{BUCKET}

gs://etsuji-15pro-poc_artifacts/


In [2]:
# Chat client to test AdkApp
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None

    async def async_stream_query(self, message):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        result = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
            print('====')
            print(event)
            print('====')
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

## Define root agent and AdkApp

In [3]:
root_agent = LlmAgent(
    name='image_analyst_agent',
    model='gemini-2.5-flash',
    instruction='''
Your role is to analyze given image files.
Use load_artifacts() if the image content is not in the context.
''',
    tools=[load_artifacts],
    planner=BuiltInPlanner(
        thinking_config=ThinkingConfig(
            include_thoughts=False,
            thinking_budget=0,
        )
    ),
)

def artifact_builder():
    return GcsArtifactService(bucket_name=BUCKET)

app = AdkApp(
    agent=root_agent,
    app_name='iamge_analyzer_app',
    artifact_service_builder=artifact_builder,
    plugins=[SaveFilesAsArtifactsPlugin()],
)

## Test the local AdkApp

In [4]:
def get_image_data(file_path: str):
    with open(file_path, 'rb') as f:
        image_bytes = f.read()
    return base64.b64encode(image_bytes).decode('utf-8')

In [32]:
client = ChatClient(app)

image_base64 = get_image_data('testimage.png')
message_input = {
        'role': 'user',
        'parts': [
            {'text': 'Count the number of people in the image.'},
            {
                'inline_data': {
                    'mime_type': 'image/png',
                    'data': image_base64
                }
            }
        ]
}

# The agent loads the image content from artifacts and insert into the prompt.
_ = await client.async_stream_query(message_input)

====
{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'function_call': {'id': 'adk-0966c56e-6661-4c75-b083-d0feda8c99a6', 'args': {'artifact_names': ['artifact_e-b3ca3385-c4b7-46d6-a22d-12105fa5351d_1']}, 'name': 'load_artifacts'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 46, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 46}], 'prompt_token_count': 1552, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 262}, {'modality': 'IMAGE', 'token_count': 1290}], 'total_token_count': 1598, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.0021358010885508165, 'invocation_id': 'e-b3ca3385-c4b7-46d6-a22d-12105fa5351d', 'author': 'image_analyst_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': 'ee593517-0faa-45d5-96c7-ac11d8f46271', 'timestamp': 1767211209.821492}
====
====
{'content': {'parts': [

In [33]:
# For the second turn, the model may use the implicit context caching  to avoid reloading the artifact.
# https://docs.cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview
_ = await client.async_stream_query('Count the number of dogs in the previous image.')

====
{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'function_call': {'id': 'adk-9e4ae8b9-2f63-49aa-a6c3-fefd80c5f5ba', 'args': {'artifact_names': ['artifact_e-b3ca3385-c4b7-46d6-a22d-12105fa5351d_1']}, 'name': 'load_artifacts'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 46, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 46}], 'prompt_token_count': 1683, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 393}, {'modality': 'IMAGE', 'token_count': 1290}], 'total_token_count': 1729, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.016883881195731785, 'invocation_id': 'e-e450f8eb-13f9-4d27-b372-1057feef5ee2', 'author': 'image_analyst_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': 'f15e0a66-692c-4c4b-9a46-014af54a6a20', 'timestamp': 1767211214.18387}
====
====
{'content': {'parts': [{'

In [34]:
# Show recorded events in the session to see that image binaries are not stored in the session.
session = await client._app.async_get_session(
    user_id = client._user_id,
    session_id = client._session_id
)
for event in session.events:
    print(event)
    print('====')

model_version=None content=Content(
  parts=[
    Part(
      text='Count the number of people in the image.'
    ),
    Part(
      text='[Uploaded Artifact: "artifact_e-b3ca3385-c4b7-46d6-a22d-12105fa5351d_1"]'
    ),
    Part(
      file_data=FileData(
        display_name='artifact_e-b3ca3385-c4b7-46d6-a22d-12105fa5351d_1',
        file_uri='gs://etsuji-15pro-poc_artifacts/iamge_analyzer_app/default_user/f34ea4c7-c4c6-4f43-94e5-c5f6c5c504f3/artifact_e-b3ca3385-c4b7-46d6-a22d-12105fa5351d_1/0',
        mime_type='image/png'
      )
    ),
  ],
  role='user'
) grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None interaction_id=None invocation_id='e-b3ca3385-c4b7-46d6-a22d-12105fa5351d' author='user' actio

## Deploy AdkApp on Agent Engine

In [36]:
agent_engines = vertexai.Client().agent_engines
display_name = 'Image Analyzer App'

remote_app = None
for item in agent_engines.list():
    if item.api_resource.display_name == display_name:
        remote_app = agent_engines.get(name=item.api_resource.name)
        break

if not remote_app:
    remote_app = agent_engines.create(
        agent=app,
        config={
            'agent_framework': 'google-adk',
            'requirements': ['google-adk==1.21.0'],
            'staging_bucket': f'gs://{PROJECT_ID}',
            'display_name': display_name,
        }
    )

The following requirements are missing: {'cloudpickle', 'pydantic', 'google-cloud-aiplatform'}


## Test the deployed AdkApp

In [37]:
client = ChatClient(remote_app)

image_base64 = get_image_data('testimage.png')
message_input = {
        'role': 'user',
        'parts': [
            {'text': 'describe the image'},
            {
                'inline_data': {
                    'mime_type': 'image/png',
                    'data': image_base64
                }
            }
        ]
}

_ = await client.async_stream_query(message_input)

====
{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'function_call': {'id': 'adk-de20f3f9-2a80-4676-9c27-faf9dd79dea5', 'args': {'artifact_names': ['artifact_e-d1aeadb6-b94f-4726-8f4b-8d28f7e7bd36_1']}, 'name': 'load_artifacts'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 45, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 45}], 'prompt_token_count': 1544, 'prompt_tokens_details': [{'modality': 'IMAGE', 'token_count': 1290}, {'modality': 'TEXT', 'token_count': 254}], 'total_token_count': 1589, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.00015677154685060182, 'invocation_id': 'e-d1aeadb6-b94f-4726-8f4b-8d28f7e7bd36', 'author': 'image_analyst_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': '69751f0e-ef46-4331-9b39-600eeb03ee5a', 'timestamp': 1767212124.350261}
====
====
{'content': {'parts': 

In [38]:
_ = await client.async_stream_query('The number of people in the image?')

====
{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'text': 'There are three people in the image.'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'cache_tokens_details': [{'modality': 'IMAGE', 'token_count': 1143}, {'modality': 'TEXT', 'token_count': 447}], 'cached_content_token_count': 1590, 'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 1795, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 505}, {'modality': 'IMAGE', 'token_count': 1290}], 'total_token_count': 1803, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.20977824926376343, 'invocation_id': 'e-de3b0fef-3342-4888-8561-3e38b55ac8a4', 'author': 'image_analyst_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': '3d945c40-fa68-488a-ba9f-d0a472f4a00a', 'timestamp': 1767212127.986427}
====
There are three people in the image.
